In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt 
from PIL import Image
import os 
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
from torchinfo import summary
import torchvision.models as models

# 전처리 & dataset 생성

In [9]:
preprocessing = transforms.Compose([
    transforms.Resize(size = (200, 200)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [10]:
file_dir = './neck'
imgDS = ImageFolder(root = file_dir, transform = preprocessing)

In [11]:
print(imgDS.classes)

['negative', 'positive']


In [12]:
imgDS.imgs[:3]

[('./neck\\negative\\normal (1).jpg', 0),
 ('./neck\\negative\\normal (10).jpg', 0),
 ('./neck\\negative\\normal (11).jpg', 0)]

In [13]:
seed = torch.Generator().manual_seed(42)
train, val, test = 0.7, 0.1, 0.2
trainDS, validDS, testDS = random_split(imgDS, [train, val, test], generator = seed)

# DataLoader

In [14]:
# 각 분류별로 dataloader를 생성해보자
batch_size = 10
train_dl = DataLoader(trainDS, batch_size = batch_size, shuffle=True)
valid_dl = DataLoader(validDS, batch_size = batch_size, shuffle=True)
test_dl = DataLoader(testDS, batch_size = batch_size, shuffle=True)

# model class 생성

In [15]:
class model(nn.Module):
    def __init__(self, IN, OUT):
        super().__init__()
        self.layer1 = nn.Conv2d(in_channels=IN, out_channels=6, kernel_size = 20, stride = 1)
                                # 채널 수           커널 수           커널사이즈       간격
        self.layer2 = nn.Conv2d(6, 12, 20 )
        self.fc = nn.Linear()

    
    def forward(self, k):
        k = self.conv(k)
        k = self.fc(k)

        return k

# 학습 위한 변수 선언

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model(2).to(device)
loss_fn = nn.BCELoss().to(device)
optimizer = optim.Adam(model.parameters())

In [24]:
model

model(
  (conv): Conv2d(3, 30, kernel_size=(20, 20), stride=(1, 1))
  (fc): Linear(in_features=30, out_features=2, bias=True)
)

In [26]:
summary(model, input_size = (1, 150, 150))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []